# Deskewing and saving images

Using recursive functions for deskewing and saving the image


In [14]:
import dask
import dask.array as da
from pyclesperanto_prototype import imshow

import pyclesperanto_prototype as cle

from aicsimageio import AICSImage, writers
from aicsimageio.types import PhysicalPixelSizes

from functools import partial
import numpy as np
import os 

from tqdm import tqdm

from skimage.io import imsave

cle.get_device()

<NVIDIA GeForce RTX 3080 on Platform: NVIDIA CUDA (2 refs)>

In [8]:
voxel_size_x_in_microns = 0.1449922
voxel_size_y_in_microns = 0.1449922
voxel_size_z_in_microns = 0.3
deskewing_angle_in_degrees = 30

In [9]:
data_path = 'C:\\RAPA_treated-01_resaved_c02_t_100.czi'
data = AICSImage(data_path)

In [10]:
image = data.dask_data
image

,Array,Chunk
Bytes,19.44 GiB,199.07 MiB
Shape,"(100, 1, 501, 140, 1488)","(1, 1, 501, 140, 1488)"
Count,500 Tasks,100 Chunks
Type,uint16,numpy.ndarray


The recursive function will call itself until it runs the function. As the function is deskewing, it returns an OCLarray. Returning as a dask array shows error and trying to save it throws an error

In [19]:
from skimage.io import imsave

save_path = "D:\\deskew_save\\"
save_name = "deskewed"

#create aicsimageio physical pixel size variable using PhysicalPixelSizes class to save pixel size
aics_image_pixel_sizes = PhysicalPixelSizes(data.physical_pixel_sizes.Z,data.physical_pixel_sizes.Y,data.physical_pixel_sizes.X)

def recursive_imageProcess(im,func,*args,**kwargs):
    if len(im.shape) > 3:
        for idx, sl in enumerate(im[0:3]):
            output=recursive_imageProcess(sl,func,*args,**kwargs)
            #Check if output is array; if so, save the file
            if type(output) is da.core.Array:
                print("Saving time: "+str(idx))
                final_name=save_path+os.sep+"C0_T"+str(idx)+"_"+save_name+".ome.tif"
                #writers.OmeTiffWriter.save(output, final_name, physical_pixel_sizes = aics_image_pixel_sizes)
                imsave(final_name, output)
                print("Saved")
        return da.asarray(output)
    return func(im,*args,**kwargs)
    

In [20]:
deskew = partial(cle.deskew_y,angle_in_degrees=deskewing_angle_in_degrees, 
                        voxel_size_x=voxel_size_x_in_microns, 
                        voxel_size_y=voxel_size_y_in_microns, 
                        voxel_size_z=voxel_size_z_in_microns)


In [21]:
%%time
output = recursive_imageProcess(image,deskew)

Saving time: 0


AssertionError: Array in get() must be contiguous

Instead, returning it as an numpy array seems to work. As clesperanto converts it to 32 bit while processing, the array is converted to 16-bit after calculation.. However, this type conversion should be handled depending on the image. 

In [22]:
from skimage.io import imsave

save_path = "D:\\deskew_save\\"
save_name = "deskewed"

#create aicsimageio physical pixel size variable using PhysicalPixelSizes class to save pixel size
aics_image_pixel_sizes = PhysicalPixelSizes(data.physical_pixel_sizes.Z,data.physical_pixel_sizes.Y,data.physical_pixel_sizes.X)

def recursive_imageProcess(im,func,*args,**kwargs):
    if len(im.shape) > 3:
        for idx, sl in enumerate(im[0:3]):
            output=recursive_imageProcess(sl,func,*args,**kwargs)
            #Check if output is array; if so, save the file
            if type(output) is np.ndarray:
                print("Saving time: "+str(idx))
                final_name=save_path+os.sep+"C0_T"+str(idx)+"_"+save_name+".ome.tif"
                #can also use aicsimageio writer
                #writers.OmeTiffWriter.save(output, final_name, physical_pixel_sizes = aics_image_pixel_sizes)
                imsave(final_name, output)
                print("Saved")
        return np.array(output).astype('uint16')
    return func(im,*args,**kwargs)

In [23]:
deskew = partial(cle.deskew_y,angle_in_degrees=deskewing_angle_in_degrees, 
                        voxel_size_x=voxel_size_x_in_microns, 
                        voxel_size_y=voxel_size_y_in_microns, 
                        voxel_size_z=voxel_size_z_in_microns)


In [24]:
%%time
output = recursive_imageProcess(image,deskew)

Saving time: 0


C:\Users\Pradeep\AppData\Local\Temp/ipykernel_23900/2511161897.py:19: UserWarning: D:\deskew_save\\C0_T0_deskewed.ome.tif is a low contrast image
  imsave(final_name, output)


Saved
Saving time: 1


C:\Users\Pradeep\AppData\Local\Temp/ipykernel_23900/2511161897.py:19: UserWarning: D:\deskew_save\\C0_T1_deskewed.ome.tif is a low contrast image
  imsave(final_name, output)


Saved
Saving time: 2


C:\Users\Pradeep\AppData\Local\Temp/ipykernel_23900/2511161897.py:19: UserWarning: D:\deskew_save\\C0_T2_deskewed.ome.tif is a low contrast image
  imsave(final_name, output)


Saved
CPU times: total: 4.02 s
Wall time: 8.18 s


In [19]:
save_path = "D:\\deskew_save\\"
save_name = "deskewed"

#create aicsimageio physical pixel size variable using PhysicalPixelSizes class to save pixel size
aics_image_pixel_sizes = PhysicalPixelSizes(data.physical_pixel_sizes.Z,data.physical_pixel_sizes.Y,data.physical_pixel_sizes.X)


#time_range = range(data.dims.T)
#we will only the first few timepoints
time_range = range(3)
channel_range = range(data.dims.C)


for time_point in tqdm(time_range, desc = "Time", position=0):    
    for ch in tqdm(channel_range, desc = "Channels", position=1,leave = False): 
        image_to_deskew = data.get_image_dask_data("ZYX",T=time_point,C=ch,S=0) #scene is zero for now
        deskewed = cle.deskew_y(image_to_deskew, angle_in_degrees = deskewing_angle_in_degrees , 
                                voxel_size_x = voxel_size_x_in_microns, voxel_size_y= voxel_size_y_in_microns, voxel_size_z = voxel_size_z_in_microns)
        final_name=save_path+os.sep+"C"+str(ch)+"_T"+str(time_point)+"_"+save_name+".ome.tif"
        #writers.OmeTiffWriter.save(deskewed, final_name, physical_pixel_sizes = aics_image_pixel_sizes)
        imsave(final_name, deskewed)

UsageError: Line magic function `%%time` not found.
